In [1]:
# Standard imports
import numpy as np
import pandas as pd
import ast 

# For scraping
import requests
from bs4 import BeautifulSoup

# For adding delays so that we don't spam requests
import time

import string

import re
import json


In [2]:

def get_genre(str_dic):
    genre_list=[]
    for i in str_dic:
        if i == '"genre":':
            j=str_dic.index(i)+1
            while str_dic[j] not in ['],','"actor":','"director":','"creator":','"datePublished":','}</script>]']:
                for punctuation_mark in string.punctuation:
                    str_dic[j]=str_dic[j].replace(punctuation_mark,'')
                if str_dic[j]!="":   
                    genre_list.append(str_dic[j])
                j+=1
            break  

    return genre_list

In [3]:
df_fifties=pd.read_csv('df_fifties.csv')
df_fifties


,tconst,primaryTitle,startYear
0,tt0025557,El negro que tenía el alma blanca,1951
1,tt0030998,You're Only Young Twice,1952
2,tt0031603,Made in Germany - Die dramatische Geschichte d...,1957
3,tt0032787,Mi noche triste,1952
4,tt0036247,Padre nuestro,1953
...,...,...,...
7794,tt7852944,Onun boyuk ureyi,1958
7795,tt7903670,Men of the Forest,1952
7796,tt8155134,Kak possorilis Ivan Ivanovich s Ivanom Nikifor...,1959
7797,tt8461508,Dick und Doof in 1000 Nöten,1958


In [4]:
scrap_df = pd.DataFrame(columns=['tconst', 'genre', 'plot'])
i=0

for tconst in df_fifties['tconst']:


    imdb_to_scrap = {'tconst':"", 'genre':[], 'plot':""}
    
    response2=requests.get(f'https://www.imdb.com/title/{tconst}/plotsummary')
    soup2 = BeautifulSoup(response2.content,'html.parser')
        
    if soup2.title.text.strip() != '404 Error - IMDb':
        plot=soup2.find('ul', id="plot-synopsis-content").text.strip()


        if plot!='It looks like we don\'t have a Synopsis for this title yet. Be the first to contribute! Just click the "Edit page" button at the bottom of the page or learn more in the Synopsis submission guide.':

            imdb_to_scrap = {'tconst':tconst, 'genre':[], 'plot':plot}

            # Send a get request and assign the response to a variable
            response1 = requests.get(f'https://www.imdb.com/title/{tconst}/')
            soup1 = BeautifulSoup(response1.content,'html.parser')


            str_dic=str(soup1.findAll('script', {'type':'application/ld+json'})).split()
            imdb_to_scrap['genre']=[get_genre(str_dic)]


            df = pd.DataFrame.from_dict(imdb_to_scrap)
            scrap_df = scrap_df.append(df, ignore_index=True)

            #follow progress: print tconst every 500 iteration
            i+=1
            if i%50==0:
                print(f'after 50 append {tconst}')


scrap_df


after 50 append tt0043465
after 50 append tt0045239
after 50 append tt0046816
after 50 append tt0048119
after 50 append tt0049169
after 50 append tt0050424
after 50 append tt0051753
after 50 append tt0053085
after 50 append tt0325507


,tconst,genre,plot
0,tt0030998,[Comedy],(The IMDb storyline is incorrect.) On the even...
1,tt0042184,"[Comedy, Romance]","Synopsis Has been Written by Mr Rais Asghar, M..."
2,tt0042192,"[Drama, contentRating, Passed]","At a prestigious awards ceremony, the celebrit..."
3,tt0042208,"[Crime, Drama, FilmNoir, Thriller]",A group of career criminals including Dix Hand...
4,tt0042249,"[Drama, War]",THE BIG LIFTThe Soviet Union withdraws from wh...
...,...,...,...
452,tt0467477,[Drama],Legendary Buddhist monk Nichiren (Kazuo Hasega...
453,tt0758020,[Drama],Three stories reflecting a socialist view of A...
454,tt1248984,"[Action, Crime]","Joji, a boxer in a world of feather and welter..."
455,tt1272042,[Drama],Ninkyo Tokaido is another great story of Yamam...


In [5]:
scrap_df

,tconst,genre,plot
0,tt0030998,[Comedy],(The IMDb storyline is incorrect.) On the even...
1,tt0042184,"[Comedy, Romance]","Synopsis Has been Written by Mr Rais Asghar, M..."
2,tt0042192,"[Drama, contentRating, Passed]","At a prestigious awards ceremony, the celebrit..."
3,tt0042208,"[Crime, Drama, FilmNoir, Thriller]",A group of career criminals including Dix Hand...
4,tt0042249,"[Drama, War]",THE BIG LIFTThe Soviet Union withdraws from wh...
...,...,...,...
452,tt0467477,[Drama],Legendary Buddhist monk Nichiren (Kazuo Hasega...
453,tt0758020,[Drama],Three stories reflecting a socialist view of A...
454,tt1248984,"[Action, Crime]","Joji, a boxer in a world of feather and welter..."
455,tt1272042,[Drama],Ninkyo Tokaido is another great story of Yamam...


In [6]:
scrap_df.to_csv ('scraped_df_fifties.csv', index = False, header=True)